In [3]:
import tensorflow as tf
import numpy as np
import netCDF4
from tensorflow.keras.utils import plot_model



In [33]:
for i in range(20,80):
  if 192%i==0:
    print('192 can be divided by {}'.format(i))
  if 175%i==0:
    print('175 can be divided by {}'.format(i))


192 can be divided by 24
175 can be divided by 25
192 can be divided by 32
175 can be divided by 35
192 can be divided by 48
192 can be divided by 64


In [74]:
# Based upon the paper by Tian. Used convLSTM instead of ConvGRU for now as the latter is not available in keras. This can later still be implemented.

def encoder(x):
  # Downsample 1a
  x = tf.keras.layers.Conv3D(filters=8, kernel_size=(1,3,3), strides=(1,2,2), padding='same', name='Downsample1a')(x)
  x = tf.keras.layers.LeakyReLU(0.2)(x)

  # Downsample 1b
  x = tf.keras.layers.Conv2D(filters=8, kernel_size=(3, 3), strides=(2,2), padding='same', name='Downsample1b')(x)
  x = tf.keras.layers.LeakyReLU(0.2)(x)   

  # RNN block 1
  x = tf.keras.layers.ConvLSTM2D(name='ConvLSTM1a', filters=64, kernel_size=(3, 3), 
                                          strides=(1,1),
                                          padding='same', 
                                          return_sequences=True)(x)
  x = tf.keras.layers.LeakyReLU(0.2)(x)

  x = tf.keras.layers.ConvLSTM2D(name='ConvLSTM1b', filters=64, kernel_size=(3, 3), 
                                          strides=(1,1),
                                          padding='same', 
                                          return_sequences=True)(x)
  x = tf.keras.layers.LeakyReLU(0.2)(x)

  # Downsample 2
  x = tf.keras.layers.Conv2D( name='Downsample2', filters=64, kernel_size=(5, 5), strides=(3,3), padding='same')(x)
  x = tf.keras.layers.LeakyReLU(0.2)(x)

  # RNN block 2
  x = tf.keras.layers.ConvLSTM2D(name='ConvLSTM2a', filters=192, kernel_size=(3, 3), 
                                          strides=(1,1),
                                          padding='same', 
                                          return_sequences=True)(x)
  x = tf.keras.layers.LeakyReLU(0.2)(x)

  x = tf.keras.layers.ConvLSTM2D(name='ConvLSTM2b', filters=192, kernel_size=(3, 3), 
                                          strides=(1,1),
                                          padding='same', 
                                          return_sequences=True)(x)
  x = tf.keras.layers.LeakyReLU(0.2)(x)  

  # Downsample 3
  x = tf.keras.layers.Conv2D( name='Downsample3', filters=192, kernel_size=(3, 3), strides=(2,2), padding='same')(x)
  x = tf.keras.layers.LeakyReLU(0.2)(x)

  # RNN block 2
  x = tf.keras.layers.ConvLSTM2D(name='ConvLSTM3a', filters=192, kernel_size=(3, 3), 
                                          strides=(1,1),
                                          padding='same', 
                                          return_sequences=True)(x)
  x = tf.keras.layers.LeakyReLU(0.2)(x)

  x = tf.keras.layers.ConvLSTM2D(name='ConvLSTM3b', filters=192, kernel_size=(3, 3), 
                                          strides=(1,1),
                                          padding='same', 
                                          return_sequences=True)(x)
  x = tf.keras.layers.LeakyReLU(0.2)(x)  
  return x

def decoder(x):
  # Decoder block 1
  x = tf.keras.layers.ConvLSTM2D(name='Decoder_Block1_ConvLSTM_1', filters=192, kernel_size=(3, 3), 
                                          strides=(1,1),
                                          padding='same', 
                                          return_sequences=True)(x)
  x = tf.keras.layers.LeakyReLU(0.2)(x)

  x = tf.keras.layers.ConvLSTM2D(name='Decoder_Block1_ConvLSTM_2', filters=192, kernel_size=(3, 3), 
                                          strides=(1,1),
                                          padding='same', 
                                          return_sequences=True)(x)
  x = tf.keras.layers.LeakyReLU(0.2)(x)  

  # Upsample 
  x = tf.keras.layers.Conv3DTranspose( name='Upsample1', filters=192, kernel_size=(1,4, 4), strides=(1,2,2), padding='same')(x)
  x = tf.keras.layers.LeakyReLU(0.2)(x)

  # Decoder block2
  x = tf.keras.layers.ConvLSTM2D(name='Decoder_Block2_ConvLSTM_1', filters=192, kernel_size=(3, 3), 
                                          strides=(1,1),
                                          padding='same', 
                                          return_sequences=True)(x)
  x = tf.keras.layers.LeakyReLU(0.2)(x)

  x = tf.keras.layers.ConvLSTM2D(name='Decoder_Block2_ConvLSTM_2', filters=192, kernel_size=(5, 5), 
                                          strides=(1,1),
                                          padding='same', 
                                          return_sequences=True)(x)
  x = tf.keras.layers.LeakyReLU(0.2)(x)  

  # Upsample 
  x = tf.keras.layers.Conv3DTranspose( name='Upsample2', filters=192, kernel_size=(1,5, 5), strides=(1,3,3), padding='same')(x)
  x = tf.keras.layers.LeakyReLU(0.2)(x)
  return x  




def model():
    input_seq = tf.keras.Input(shape=(5, 768, 700, 1))

    x = encoder(input_seq)
    x = decoder(x)
    # output = tf.keras.layers.Conv2D(filters=1, kernel_size=(1, 1), 
    #                                 strides=(1,1), padding='same')(convLSTM)
    
    # output = tf.keras.layers.Reshape(target_shape=(1,384, 350, 1))(output)
        
    model = tf.keras.Model(inputs=input_seq, outputs=x)
    return model

model = model()
model.summary()
#plot_model(model)
model.compile( optimizer='sgd',
    loss='mse',
    metrics=[tf.keras.metrics.MeanSquaredError()])


Model: "model_41"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_57 (InputLayer)        [(None, 5, 768, 700, 1)]  0         
_________________________________________________________________
Downsample1a (Conv3D)        (None, 5, 384, 350, 8)    80        
_________________________________________________________________
leaky_re_lu_378 (LeakyReLU)  (None, 5, 384, 350, 8)    0         
_________________________________________________________________
Downsample1b (Conv2D)        (None, 5, 192, 175, 8)    584       
_________________________________________________________________
leaky_re_lu_379 (LeakyReLU)  (None, 5, 192, 175, 8)    0         
_________________________________________________________________
ConvLSTM1a (ConvLSTM2D)      (None, 5, 192, 175, 64)   166144    
_________________________________________________________________
leaky_re_lu_380 (LeakyReLU)  (None, 5, 192, 175, 64)   0  

In [ ]:

def get_mask_y():
    # Get the mask for the input data
    y_path = '/nobackup/users/schreurs/project_GAN/dataset_aart'
    # The mask is the same for all radar scans, so simply chose a random one to get the mask
    path = y_path + '/RAD_NL25_RAC_MFBS_EM_5min_201901010000.nc'

    with netCDF4.Dataset(path, 'r') as f:
        rain = f['image1_image_data'][:]
        mask = (rain == 65535)
    mask = mask.astype(float)
    mask = np.expand_dims(mask, axis=0)
    mask = np.expand_dims(mask, axis=-1)

    mask = crop_center(mask)
    return mask

def crop_center(img,cropx=350,cropy=384):
    # batch size, sequence, height, width, channels
     # Only change height and width
    _,_, y,x, _ = img.shape
    startx = 20+x//2-(cropx//2)
    starty = 40+y//2-(cropy//2)    
    return img[:,:,starty:starty+cropy,startx:startx+cropx:,]

def model():
    input_seq = tf.keras.Input(shape=(5, 768, 700, 1))

    down = tf.keras.layers.Conv2D(filters=8, kernel_size=(5, 5), strides=(2,2), padding='same')(input_seq)
    down = tf.keras.layers.LeakyReLU()(down)
    
    convLSTM = tf.keras.layers.ConvLSTM2D(filters=64, kernel_size=(3, 3), 
                                          strides=(1,1),
                                          padding='same', 
                                          return_sequences=True)(convLSTM)
    
    convLSTM = tf.keras.layers.ConvLSTM2D(filters=64, kernel_size=(3, 3), 
                                          strides=(1,1),
                                          padding='same', 
                                          return_sequences=True)(convLSTM)
    
    output = tf.keras.layers.Conv2D(filters=1, kernel_size=(1, 1), 
                                    strides=(1,1), padding='same')(up)
    
    output = tf.keras.layers.Reshape(target_shape=(1,765, 700, 1))(output)
    
    output_with_mask = tf.keras.layers.Multiply()([down, get_mask_y()])
    
    model = tf.keras.Model(inputs=input_seq, outputs=output_with_mask)
    return model

model = model()
model.summary()
#plot_model(model)
model.compile( optimizer='sgd',
    loss='mse',
    metrics=[tf.keras.metrics.MeanSquaredError()])
